In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [ ]:
user_question = "I just discovered the course. Can I still join it?"

In [ ]:
v = embedding_model.encode(user_question)
v[0]

In [ ]:
import requests
from tqdm.auto import tqdm

docs_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for document in tqdm(documents_raw):
    if document['course'] == 'machine-learning-zoomcamp':
        documents.append(document)

In [ ]:
len(documents)

In [ ]:
from tqdm.auto import tqdm
import numpy as np

In [ ]:
embeddings = []

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    emb = embedding_model.encode(qa_text)
    embeddings.append(emb)

In [ ]:
X = np.array(embeddings)
X.shape

In [ ]:
scores = X.dot(v)
scores.max()

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

In [ ]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    v_doc_question = embedding_model.encode(q['question'])
    results = search_engine.search(v_doc_question, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [ ]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [ ]:
hit_rate(relevance_total)

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
i = 0

for doc in tqdm(documents):
    
    doc['question_text_vector'] = X[i, :]
    i = i + 1

    es_client.index(index=index_name, document=doc)

In [ ]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
elastic_search_knn("question_text_vector", v, "machine-learning-zoomcamp")

In [ ]:
def vector_question_text_elastic(q):
    question = q['question']
    
    v_q = embedding_model.encode(question)

    return elastic_search_knn("question_text_vector", v_q, "machine-learning-zoomcamp")

In [ ]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

In [ ]:
evaluate(ground_truth, vector_question_text_elastic)